In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

from tqdm import tqdm
import akshare as ak
import datetime
from multiprocessing import Pool
from sklearn.metrics import confusion_matrix

ModuleNotFoundError: No module named 'akshare'

In [2]:
def MA_strategy(stockcode, show=0):
    # 导入股票
    try:
        stock_code = ak.stock_zh_a_hist(symbol=stockcode, period="daily", start_date='20200101', end_date='20250306', adjust="hfq")[['日期','股票代码','开盘','收盘','最高','最低','成交量','成交额','换手率']]
    except:
        return stockcode, np.nan, np.nan, np.nan, np.nan
    if len(stock_code)<100:
        return stockcode, np.nan, np.nan, np.nan, np.nan
    if (stock_code['收盘'] < 0).any():
        return stockcode, np.nan, np.nan, np.nan, np.nan
    
    stock_code.rename(columns={'日期':'date','股票代码':'stockcode','成交量':'turnovervolumn','成交额':'turmovervalue','换手率':'turnoverrate',
                                '开盘':'backward_openprice','收盘':'backward_closeprice','最高':'backward_highprice','最低':'backward_lowprice'},inplace=True)
    stock_code['date'] = stock_code['date'].astype(str)
    stock_code.set_index(['date','stockcode'],inplace=True)
    stock_code.sort_index(inplace=True)

    # 指标计算
    stock_code['MA_5'] = stock_code['backward_closeprice'].rolling(window=5,min_periods=5).mean()
    stock_code['MA_20'] = stock_code['backward_closeprice'].rolling(window=20,min_periods=20).mean()

    # 策略回测
    fund = 1
    hold = 0
    fund_list = []
    hold_list = []
    date_list = []
    sign_list = []

    for date in stock_code.index:
        if pd.isna(stock_code.loc[date, 'MA_20']):
            continue

        if stock_code.loc[date,'MA_5'] > stock_code.loc[date,'MA_20']:
            if hold == 0:
                fund = fund
                hold = fund / stock_code.loc[date,'backward_closeprice']
                fund_list.append(fund)
                hold_list.append(hold)
                date_list.append(date[0])
                sign_list.append(1)
                continue

            if hold != 0:
                fund = hold * stock_code.loc[date,'backward_closeprice']
                hold = hold
                fund_list.append(fund)
                hold_list.append(hold)
                date_list.append(date[0])
                sign_list.append(0)
                continue

        if stock_code.loc[date,'MA_5'] <= stock_code.loc[date,'MA_20']:
            if hold == 0:
                fund = fund
                hold = 0
                fund_list.append(fund)
                hold_list.append(hold)
                date_list.append(date[0])
                sign_list.append(0)
                continue

            if hold != 0:
                fund = hold * stock_code.loc[date,'backward_closeprice']
                hold = 0
                fund_list.append(fund)
                hold_list.append(hold)
                date_list.append(date[0])
                sign_list.append(-1)
                continue

    # 计算回测收益
    PnL = pd.DataFrame({'strategy':fund_list,'hold':hold_list,'sign':sign_list},  pd.MultiIndex.from_arrays([date_list, [stockcode] * len(date_list)],  names=['date', 'stock_code']))
    PnL['price'] = stock_code['backward_closeprice']
    PnL['price'] = PnL['price']/ PnL.loc[PnL.index[0],'price']

    # 计算回测指标
    returns = PnL['strategy'].pct_change().dropna()  # 计算每日收益率
    annual_return = (PnL['strategy'].iloc[-1] / PnL['strategy'].iloc[0]) ** (252 / len(PnL)) - 1
    annual_volatility = returns.std() * np.sqrt(252)
    cum_max = PnL['strategy'].cummax()  # 计算累计最大净值
    drawdown = PnL['strategy'] / cum_max - 1  # 计算回撤
    max_drawdown = drawdown.min()  # 取最大回撤
    sharpe_ratio = annual_return / annual_volatility if annual_volatility != 0 else np.nan

    if show == 1:
        # 策略结果可视化
        fig, ax1 = plt.subplots(figsize=(20, 6))
        df = PnL.copy(deep=True)
        df.index = pd.MultiIndex.from_tuples([(pd.to_datetime(date), stock) for date, stock in df.index], names=['date', 'stock_code'])

        # 折线图
        ax1.plot(df.index.get_level_values('date'), df['price'], label='price', color='tab:blue', linewidth=1)
        ax1.plot(df.index.get_level_values('date'), df['strategy'], label='strategy', color='tab:green', linestyle='dashed')

        # 添加买卖点
        buy_signals = df[df['sign'] == 1]
        sell_signals = df[df['sign'] == -1]
        ax1.scatter(buy_signals.index.get_level_values('date'), buy_signals['price'], color='red', label='buy', marker='o', s=15)
        ax1.scatter(sell_signals.index.get_level_values('date'), sell_signals['price'], color='blue', label='sell', marker='o', s=15)

        # 添加文本信息
        textstr = f"annual_return: {annual_return:.2%}\nannual_volatility: {annual_volatility:.2%}\nmax_drawdown: {max_drawdown:.2%}\nsharpe_ratio: {sharpe_ratio:.2f}"
        ax1.text(df.index.get_level_values('date')[-int(len(df) * 0.2)], df['price'].max() * 0.9, textstr,
                fontsize=12, bbox=dict(facecolor='white', alpha=0.6))

        ax1.set_xlabel('date')
        ax1.set_ylabel('net value')
        ax1.set_title(f'stockcode {PnL.index[0][1]} return')
        ax1.legend()
        plt.grid(alpha=0.5)
        plt.show()

    return stockcode,annual_return, annual_volatility, max_drawdown, sharpe_ratio

In [3]:
# 获取股票列表
today_stock = ak.stock_zh_a_spot_em()[['代码']]
stock_code_list = sorted(today_stock['代码'].to_list())

# 并行运算策略
with Pool() as pool:
    results = list(tqdm(pool.imap(MA_strategy, stock_code_list), total=len(stock_code_list)))

# 保存策略结果
stock_code_list = []
stock_annual_return = []    
stock_annual_volatility = []
stock_max_drawdown = []
stock_sharpe_ratio =[]
for data in results:
    stock_code_list.append(data[0])
    stock_annual_return.append(data[1])
    stock_annual_volatility.append(data[2])
    stock_max_drawdown.append(data[3])
    stock_sharpe_ratio.append(data[4])

100%|██████████| 5697/5697 [00:59<00:00, 95.07it/s] 


In [4]:
# 策略结果分析
MA_strategy_result = pd.DataFrame(
    {
        'stockcode':stock_code_list,
        'annual_return':stock_annual_return,
        'annual_volatility':stock_annual_volatility,
        'max_drawdown':stock_max_drawdown,
        'sharpe_ratio':stock_sharpe_ratio,
    }
)
MA_strategy_result.dropna(inplace=True)
MA_strategy_result.set_index(['stockcode'],inplace=True)

In [12]:
MA_strategy_result.mean()

annual_return        0.026607
annual_volatility    0.367432
max_drawdown        -0.508838
sharpe_ratio         0.015856
dtype: float64